# APIM ❤️ OpenAI

## Zero-to-Production lab

Playground to create a combination of several policies in an iterative approach. We will accomplish three successive policy additions:

1) Add load balancing across multiple region
1) Add token emitting to observe token usage
1) Apply token rate limiting to avoid runaway token usage scenarios

Each of these sets of policies is derived from other labs in this repo.

### Prerequisites
- [Python 3.12 or later version](https://www.python.org/) installed
- [Pandas Library](https://pandas.pydata.org/) and matplotlib installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access) or just enable the mock service
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/cli/azure/authenticate-azure-cli-interactively)

### Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id.
- Change the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)
- Adjust the OpenAI model, version according the [availability by region.](https://learn.microsoft.com/azure/ai-services/openai/concepts/models) 
- Experiment with the priority, weight, and capacity OpenAI parameters to affect the load balancing
- `capacity` is set intentionally low - the unit is tokens per minute - to trigger the retry logic in the load balancer (transparent to the user) as well as the priority failover from priority 1 to 2.

In [ ]:
import os, sys, json
sys.path.insert(1, '../../shared')  # add the shared directory to the Python path
import utils

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "eastus2"

apim_sku = 'Basicv2'

# Prioritize East US until exhaustion (simulate PTU with TPM), then equally distribute between Sweden and West US (consumption fallback)
openai_resources = [
    {"name": "openai1", "location": "eastus", "priority": 1, "weight": 100, "capacity": 4},
    {"name": "openai2", "location": "swedencentral", "priority": 2, "weight": 50, "capacity": 8 },
    {"name": "openai3", "location": "westus", "priority": 2, "weight": 50, "capacity": 8}
]

openai_deployment_name = "gpt-4o-mini"
openai_model_name = "gpt-4o-mini"
openai_model_version = "2024-07-18"
openai_model_sku = 'Standard'
openai_api_version = "2024-02-01"

backend_id = 'openai-backend-pool' if len(openai_resources) > 1 else openai_resources[0]['name']

utils.print_ok('Notebook initialized')

### Verify the Azure CLI and the connected Azure subscription

The following commands ensure that you have the latest version of the Azure CLI and that the Azure CLI is connected to your Azure subscription.

In [ ]:
output = utils.run("az account show", "Retrieved az account", "Failed to get the current az account")

if output.success and output.json_data:
    current_user = output.json_data['user']['name']
    tenant_id = output.json_data['tenantId']
    subscription_id = output.json_data['id']

    utils.print_info(f"Current user: {current_user}")
    utils.print_info(f"Tenant ID: {tenant_id}")
    utils.print_info(f"Subscription ID: {subscription_id}")

### Policy 1 - Load Balancing

This lab uses [Bicep](https://learn.microsoft.com/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declaratively define all resources that will be deployed in the specified resource group. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations.

#### Create deployment using 🦾 Bicep

The `retry-count` parameter should have a value that represents one less than the total number of backends. For example, if we have three defined Azure OpenAI backends, we want to try initially, then have up to two retries, so long as we have remaining, active backends. This ensures that we cover all available backends.

In [ ]:
policy_xml_file = "policy-1.xml"
bicep_parameters_file = "params-1.json"

# Create the resource group if doesn't exist
utils.create_resource_group(resource_group_name, resource_group_location)

# Define the Bicep parameters
bicep_parameters = {
    "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
    "contentVersion": "1.0.0.0",
    "parameters": {
        "apimSku": { "value": apim_sku },
        "openAIConfig": { "value": openai_resources },
        "openAIDeploymentName": { "value": openai_deployment_name },
        "openAIModelName": { "value": openai_model_name },
        "openAIModelVersion": { "value": openai_model_version },
        "openAIModelSKU": { "value": openai_model_sku },
        "openAIAPIVersion": { "value": openai_api_version }
    }
}

bicep_parameters = utils.create_bicep_params(policy_xml_file, bicep_parameters_file, bicep_parameters, [
    ('{backend-id}', backend_id),
    ('{retry-count}', len(openai_resources) - 1)
])

# Run the deployment
output = utils.run(f"az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file main.bicep --parameters {bicep_parameters_file}",
    f"Deployment '{deployment_name}' succeeded", f"Deployment '{deployment_name}' failed")

<a id='2deploymentoutputs'></a>
#### Get the deployment outputs

Retrieve the required outputs from the Bicep deployment.

In [ ]:
# Obtain all of the outputs from the deployment
output = utils.run(f"az deployment group show --name {deployment_name} -g {resource_group_name}", f"Retrieved deployment: {deployment_name}", f"Failed to retrieve deployment: {deployment_name}")

if output.success and output.json_data:
    apim_service_id = utils.get_deployment_output(output, 'apimServiceId', 'APIM Service Id')
    apim_service_name = utils.get_deployment_output(output, 'apimServiceName', 'APIM Service Name')
    apim_resource_gateway_url = utils.get_deployment_output(output, 'apimResourceGatewayURL', 'APIM API Gateway URL')
    apim_subscription1_key = utils.get_deployment_output(output, 'apimSubscription1Key', 'APIM Subscription 1 Key (masked)', True)
    apim_subscription2_key = utils.get_deployment_output(output, 'apimSubscription2Key', 'APIM Subscription 2 Key (masked)', True)
    apim_subscription3_key = utils.get_deployment_output(output, 'apimSubscription3Key', 'APIM Subscription 3 Key (masked)', True)
    app_insights_name = utils.get_deployment_output(output, 'applicationInsightsName', 'Application Insights Name')

<a id='2sdk'></a>
#### 🧪 Test the API using the Azure OpenAI Python SDK

Use the OpenAI Python SDK to make requests to API Management and any of the Azure OpenAI backends. Note that we return the `x-ms-region` header to show the frontend which backend was used. You may not want to do that in a production scenario.

You will not see HTTP 429s returned as API Management's `retry` policy will select an available backend. If no backends are viable, an HTTP 503 will be returned.

Tip: Use the [tracing tool](../../tools/tracing.ipynb) to track the behavior of the backend pool.

In [ ]:
import time
from openai import AzureOpenAI

runs = 20
sleep_time_ms = 100
total_tokens_all_runs = 0
api_runs = []

client = AzureOpenAI(
    azure_endpoint = apim_resource_gateway_url,
    api_key = apim_subscription1_key,
    api_version = openai_api_version
)

for i in range(runs):
    print(f"▶️ Run {i+1}/{runs}:")

    start_time = time.time()
    raw_response = client.chat.completions.with_raw_response.create(
        model = openai_model_name,
        messages = [
            {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
            {"role": "user", "content": "Can you tell me the time, please?"}
        ])
    response_time = time.time() - start_time

    print(f"⌚ {response_time:.2f} seconds")

    if "x-ms-region" in raw_response.headers:
        print(f"x-ms-region: \x1b[1;32m{raw_response.headers.get("x-ms-region")}\x1b[0m") # this header is useful to determine the region of the backend that served the request
        api_runs.append((response_time, raw_response.headers.get("x-ms-region")))

    response = raw_response.parse()

    if response.usage:
        total_tokens_all_runs += response.usage.total_tokens
        print(f"Token usage:\n   Total tokens: {response.usage.total_tokens}\n   Prompt tokens: {response.usage.prompt_tokens}\n   Completion tokens: {response.usage.completion_tokens}\n   Total tokens all runs: {total_tokens_all_runs}\n")


    print(f"💬 {response.choices[0].message.content}\n")

    time.sleep(sleep_time_ms/1000)

<a id='2plot'></a>
#### 🔍 Analyze Load Balancing results

The priority 1 backend will be used until TPM exhaustion sets in, then distribution will occur near equally across the two priority 2 backends with 50/50 weights.  

Please note that the first request of the lab can take a bit longer and should be discounted statistically as an outlier.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle as pltRectangle
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = [15, 7]
df = pd.DataFrame(api_runs, columns = ['Response Time', 'Region'])
df['Run'] = range(1, len(df) + 1)

# Define a color map for each region
color_map = {'East US': 'lightpink', 'Sweden Central': 'lightyellow', 'West US': 'lightblue'}  # Add more regions and colors as needed

# Plot the dataframe with colored bars
ax = df.plot(kind = 'bar', x = 'Run', y = 'Response Time', color = [color_map.get(region, 'gray') for region in df['Region']], legend = False)

# Add legend
legend_labels = [pltRectangle((0, 0), 1, 1, color = color_map.get(region, 'gray')) for region in df['Region'].unique()]
ax.legend(legend_labels, df['Region'].unique())

plt.title('Load Balancing results')
plt.xlabel('Run #')
plt.ylabel('Response Time')
plt.xticks(rotation = 0)

average = df['Response Time'].mean()
plt.axhline(y = average, color = 'r', linestyle = '--', label = f'Average: {average:.2f}')

plt.show()

### Policy 2 - Token Emitting

We now add token emitting to the existing API policy in order to track token usage by subscriptions. This aids usage and cost analysis and chargeback models inside organizations. You can see the policy to be added in the `policy-2.xml` file in this folder.

#### Update the API management policy via the REST API

In [ ]:
policy_xml_file = "policy-2.xml"

with open(policy_xml_file, 'r') as file:
    policy_xml = file.read()
    policy_xml = policy_xml.replace('{backend-id}', backend_id).replace('{retry-count}', str(len(openai_resources) - 1))

utils.update_api_policy(subscription_id, resource_group_name, apim_service_name, "openai", policy_xml)

<a id='3sdk'></a>
#### 🧪 Execute multiple runs for each subscription using the Azure OpenAI Python SDK

We will send requests for each subscription. Adjust the `sleep_time_ms` and the number of `runs` to your test scenario.


In [ ]:
import time
from openai import AzureOpenAI

runs = 5
sleep_time_ms = 100
total_tokens_all_runs = [0, 0, 0]

clients = [
    AzureOpenAI(
        azure_endpoint = apim_resource_gateway_url,
        api_key = apim_subscription1_key,
        api_version = openai_api_version
    ),
    AzureOpenAI(
        azure_endpoint = apim_resource_gateway_url,
        api_key = apim_subscription2_key,
        api_version = openai_api_version
    ),
    AzureOpenAI(
        azure_endpoint = apim_resource_gateway_url,
        api_key = apim_subscription3_key,
        api_version = openai_api_version
    )
]

for i in range(runs):
    print(f"▶️ Run {i+1}/{runs}:")

    for j in range(0, 3):
        start_time = time.time()

        try:
            raw_response = clients[j].chat.completions.with_raw_response.create(
                model = openai_model_name,
                messages = [
                    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
                    {"role": "user", "content": "Can you tell me the time, please?"}
                ],
                extra_headers = {"x-user-id": "alex"}
            )

            response_time = time.time() - start_time
            print(f"🔑 Subscription {j+1}")
            print(f"⌚ {response_time:.2f} seconds")

            if "x-ms-region" in raw_response.headers:
                print(f"x-ms-region: \x1b[1;32m{raw_response.headers.get("x-ms-region")}\x1b[0m") # this header is useful to determine the region of the backend that served the request

            response = raw_response.parse()

            if response.usage:
                total_tokens_all_runs[j] += response.usage.total_tokens
                print(f"Token usage:\n   Total tokens: {response.usage.total_tokens}\n   Prompt tokens: {response.usage.prompt_tokens}\n   Completion tokens: {response.usage.completion_tokens}\n   Total tokens all runs: {total_tokens_all_runs[j]}\n")

            print(f"💬 {response.choices[0].message.content}\n")
        except Exception as e:
            print(e)

    print()

    time.sleep(sleep_time_ms/1000)


<a id='3metricsinportal'></a>
### 🔍 See the metrics on the Azure Portal

One way to see the newly-added token metrics in the Azure Portal:

1) Open the _Application Insights_ resource in the resource group.
1) Navigate to the _Metrics_ blade.
1) Change the timespan to the last 30 minutes with a 1 minute time granularity.
1) Then select the _openai_ metric namespace. 
1) Choose the _Total Tokens_ metric 
1) Select the _Sum_ aggregation. 
1) Apply splitting by _Subscription Id_ to view values for each dimension. For better visibility switch to an area chart.

![result](result.png)


<a id='3kql'></a>
#### 🔍 Analyze Application Insights custom metrics with a KQL query

Alternatively, you can query the custom metrics that were emitted by Azure APIM. **Note that it may take a few minutes for data to become available.** If you see the metrics in the Azure Portal, you don't have to wait for the data here and plotting it to continue. Come back to this section later, if you like.

In [ ]:
import pandas as pd

query = "\"" + "customMetrics \
| where name == 'Total Tokens' \
| extend parsedCustomDimensions = parse_json(customDimensions) \
| extend clientIP = tostring(parsedCustomDimensions.['Client IP']) \
| extend apiId = tostring(parsedCustomDimensions.['API ID']) \
| extend apimSubscription = tostring(parsedCustomDimensions.['Subscription ID']) \
| extend UserId = tostring(parsedCustomDimensions.['User ID']) \
| project timestamp, value, clientIP, apiId, apimSubscription, UserId \
| order by timestamp asc" + "\""

output = utils.run(f"az monitor app-insights query --app {app_insights_name} -g {resource_group_name} --analytics-query {query}",
    f"App Insights query succeeded", f"App Insights query  failed")

table = output.json_data['tables'][0]
df = pd.DataFrame(table.get("rows"), columns = [col.get("name") for col in table.get('columns')])
df['timestamp'] = pd.to_datetime(df['timestamp']).dt.strftime('%H:%M')
df

<a id='3plot'></a>
#### 🔍 Plot the custom metrics results

In [ ]:
# plot the results
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = [15, 7]
ax = df.plot(kind = 'line', x = 'timestamp', y = 'value', legend = False)
plt.title('Total token usage over time')
plt.xlabel('Time')
plt.ylabel('Tokens')

plt.show()

### Policy 3 - Token Rate Limiting

Emitting tokens is very valuable to understand usage. Even more valuable is adding rate limiting for subscriptions as a sensible way to limit runaway usage.

Please note that results are going to be a bit skewed as we are executing these requests sequentially, but it should get the gist across as to what's happening from a rate-limiting perspective. Threading may be considered in the future.

#### Update the API management policy via the REST API

In [ ]:
policy_xml_file = "policy-3.xml"
tokens_per_minute = 500

with open(policy_xml_file, 'r') as file:
    policy_xml = file.read()
    policy_xml = policy_xml.replace('{backend-id}', backend_id).replace('{retry-count}', str(len(openai_resources) - 1)).replace('{tpm}', str(tokens_per_minute))

utils.update_api_policy(subscription_id, resource_group_name, apim_service_name, "openai", policy_xml)

<a id='4sdk'></a>
#### 🧪 Execute multiple runs for each subscription using the Azure OpenAI Python SDK

We will send requests for each subscription. Adjust the `sleep_time_ms` and the number of `runs` to your test scenario. You should be able to observe a significant pause after a few runs as the requests will hit the tokens-per-minute limit that we defined earlier. This is expected and validation of the policies working.


In [ ]:
import time
from openai import AzureOpenAI

runs = 3
sleep_time_ms = 100
total_tokens_all_runs = [0, 0, 0]
api_runs = []

clients = [
    AzureOpenAI(
        azure_endpoint = apim_resource_gateway_url,
        api_key = apim_subscription1_key,
        api_version = openai_api_version
    ),
    AzureOpenAI(
        azure_endpoint = apim_resource_gateway_url,
        api_key = apim_subscription2_key,
        api_version = openai_api_version
    ),
    AzureOpenAI(
        azure_endpoint = apim_resource_gateway_url,
        api_key = apim_subscription3_key,
        api_version = openai_api_version
    )
]

for i in range(runs):
    print(f"▶️ Run {i+1}/{runs}:")

    for j in range(0, 3):
        start_time = time.time()

        try:
            raw_response = clients[j].chat.completions.with_raw_response.create(
                model = openai_model_name,
                messages = [
                    {"role": "system", "content": "You are a sarcastic, unhelpful assistant."},
                    {"role": "user", "content": "Can you tell me the time, please?"}
                ],
                extra_headers = {"x-user-id": "alex"}
            )

            response_time = time.time() - start_time
            print(f"🔑 Subscription {j+1}")
            print(f"⌚ {response_time:.2f} seconds")
            api_runs.append((response_time, j+1))

            if "x-ms-region" in raw_response.headers:
                print(f"x-ms-region: \x1b[1;32m{raw_response.headers.get("x-ms-region")}\x1b[0m") # this header is useful to determine the region of the backend that served the request

            response = raw_response.parse()

            if response.usage:
                total_tokens_all_runs[j] += response.usage.total_tokens
                print(f"Token usage:\n   Total tokens: {response.usage.total_tokens}\n   Prompt tokens: {response.usage.prompt_tokens}\n   Completion tokens: {response.usage.completion_tokens}\n   Total tokens all runs: {total_tokens_all_runs[j]}\n")

            print(f"💬 {response.choices[0].message.content}\n")
        except e as Exception:
            print(e)

    print()

    time.sleep(sleep_time_ms/1000)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle as pltRectangle
import matplotlib as mpl

mpl.rcParams['figure.figsize'] = [15, 7]
df = pd.DataFrame(api_runs, columns = ['Response Time', 'Subscription'])
df['Run'] = range(1, len(df) + 1)

# Define a color map for each subscription
color_map = {1: 'lightpink', 2: 'lightyellow', 3: 'lightblue'}  # Add more subscriptions and colors as needed

# Plot the dataframe with colored bars
ax = df.plot(kind = 'bar', x = 'Run', y = 'Response Time', color = [color_map.get(subscription, 'gray') for subscription in df['Subscription']], legend = False)

# Add legend
legend_labels = [pltRectangle((0, 0), 1, 1, color = color_map[subscription]) for subscription in color_map.keys()]
ax.legend(legend_labels, ['Subscription 1', 'Subscription 2', 'Subscription 3'])

plt.title('Token Rate Limiting by Subscription Results (high response times indicate likely throttling)')
plt.xlabel('Run #')
plt.ylabel('Response Time')
plt.xticks(rotation = 0)

average = df['Response Time'].mean()
plt.axhline(y = average, color = 'r', linestyle = '--', label = f'Average: {average:.2f}')

plt.show()

<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.